## バフェットの財務諸表を読む力で書いてある事をQuantopianで解く

[史上最強の投資家 バフェットの財務諸表を読む力 大不況でも投資で勝ち抜く58のルール | メアリー・バフェット, デビッド・クラーク, 峯村利哉 |本 | 通販 | Amazon](https://www.amazon.co.jp/dp/4198627053)


### 粗利益率
+ 粗利益/売上高
+ 一貫して高い率が良い．（例KO：60％あたり）
+ [gross-profit](https://www.quantopian.com/docs/data-reference/morningstar_fundamentals#gross-profit)
    + おそらくQuartary
+ [sales](https://www.quantopian.com/docs/data-reference/factset_fundamentals#sales-af-sales-ltm-sales-qf-sales-saf)
    + sales_af, sales_ltm, sales_qf, sales_saf
    


### 金利コスト/粗利

+ 営業利益に占める支払利息の比率＝企業の危機レベル

In [ ]:
import numpy as np

# Import Pipeline class and datasets
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.data.morningstar import Fundamentals
from quantopian.pipeline.data.factset import Fundamentals as factst_funda
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets

# Import built-in trading universe
from quantopian.pipeline.filters import QTradableStocksUS


def make_pipeline():
    # Create a reference to our trading universe
    base_universe = QTradableStocksUS()

    # Get latest closing price
    close_price = USEquityPricing.close.latest
    
    # 売上高
    gross_profit = Fundamentals.gross_profit.latest
    # 粗利益率
    sales_qf = factst_funda.sales_qf.latest
    # 粗利益率
    sales_profit_ratio = sales_qf / gross_profit
    
    
    
    # interest_expense
    interest_expense = Fundamentals.interest_expense.latest
    morningstar_sector_code = Fundamentals.morningstar_sector_code.latest
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest

    myassets = StaticAssets(symbols(['AAPL', 'FB', 'MSFT', 'AMZN', 'GOOG', 'KO']))
    
    # Return Pipeline containing close_price and
    # sentiment_score that has our trading universe as screen
    return Pipeline(
        columns={
            'close_price': close_price,
            'interest_expense':interest_expense,
            'gross_profit':gross_profit, 
            'morningstar_sector_code':morningstar_sector_code, 
            'morningstar_industry_group_code':morningstar_industry_group_code, 
            'sales_profit_ratio':sales_profit_ratio,
        },
        screen=base_universe #myassets
    )

# Import run_pipeline method
from quantopian.research import run_pipeline

# Execute pipeline created by make_pipeline
# between start_date and end_date
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date='2019-01-01',
    end_date='2020-04-01'
)

# Display last 10 rows
pipeline_output.tail(10)

In [ ]:
pipeline_output["interest/profit"] = pipeline_output["interest_expense"]/pipeline_output["gross_profit"]

In [ ]:
# 興味があったのでバイオテクノロジーだけやってみる
biotech = pipeline_output[pipeline_output["morningstar_industry_group_code"]==20610]


In [ ]:
biotech=biotech.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

In [ ]:
biotech = biotech["close_price"].unstack().dropna(axis=1)

In [ ]:
x = (biotech.pct_change() + 1 ).cumprod()

In [ ]:
syms = x.iloc[-1].sort_values().tail(5).index 

In [ ]:
x[syms].plot()